<a href="https://colab.research.google.com/github/kevinmastascusa/symbolic-math-ai/blob/main/01_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# Install required packages using magic commands
import sys
!{sys.executable} -m pip install pandas>=1.5.0 numpy>=1.21.0 matplotlib>=3.5.0 seaborn>=0.11.0 scikit-learn>=1.1.0 requests huggingface_hub --quiet
!{sys.executable} -m pip install --upgrade -q datasets

print("✅ Packages installed successfully!")



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


✅ Packages installed successfully!



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
# Download and save real datasets for GSM8K, MathQA, and MAWPS
import pandas as pd
import requests
from pathlib import Path

# def download_mawps():
    # try:
    #     print("Downloading MAWPS dataset...")
    #     # Try multiple MAWPS URLs since the original might be down
    #     urls = [
    #         'https://raw.githubusercontent.com/wang-research-lab/regal/main/data/mawps.json',
    #         'https://raw.githubusercontent.com/allenai/mawps/master/data/mawps_no_anonymized.json'
    #     ]

    #     data = None
    #     for url in urls:
    #         try:
    #             response = requests.get(url, timeout=30)
    #             response.raise_for_status()
    #             data = response.json()
    #             print(f"Successfully fetched from: {url}")
    #             break
    #         except:
    #             continue

    #     if data is None:
    #         # Create sample MAWPS data if download fails
    #         print("Creating sample MAWPS data...")
    #         data = [
    #             {'sQuestion': 'A train travels 120 miles in 2 hours. What is its speed?', 'lSolutions': ['60'], 'lEquations': ['120/2']},
    #             {'sQuestion': 'John has 15 apples. He gives 8 to Mary. How many does he have left?', 'lSolutions': ['7'], 'lEquations': ['15-8']},
    #             {'sQuestion': 'A rectangle has length 12 and width 8. What is its area?', 'lSolutions': ['96'], 'lEquations': ['12*8']},
    #             {'sQuestion': 'Sarah buys 5 packs of gum. Each pack has 12 pieces. How many pieces total?', 'lSolutions': ['60'], 'lEquations': ['5*12']},
    #             {'sQuestion': 'There are 28 students. 15 are girls. How many are boys?', 'lSolutions': ['13'], 'lEquations': ['28-15']},
    #             {'sQuestion': 'A car travels 180 miles in 3 hours. What is its speed?', 'lSolutions': ['60'], 'lEquations': ['180/3']},
    #             {'sQuestion': 'Mike has 24 stickers. He divides them equally among 4 friends. How many does each get?', 'lSolutions': ['6'], 'lEquations': ['24/4']},
    #             {'sQuestion': 'A box contains 36 chocolates arranged in 6 rows. How many chocolates per row?', 'lSolutions': ['6'], 'lEquations': ['36/6']},
    #             {'sQuestion': 'Lisa saves $5 per week for 8 weeks. How much does she save total?', 'lSolutions': ['40'], 'lEquations': ['5*8']},
    #             {'sQuestion': 'A pizza is cut into 8 slices. Tom eats 3 slices. How many are left?', 'lSolutions': ['5'], 'lEquations': ['8-3']}
    #         ]

    #     problems = []
    #     for i, item in enumerate(data):
    #         problems.append({
    #             'sQuestion': item.get('sQuestion', f'Problem {i+1}'),
    #             'lSolutions': item.get('lSolutions', ['0']),
    #             'lEquations': item.get('lEquations', []),
    #             'iIndex': item.get('iIndex', i),
    #             'category': item.get('category', 'word_problem'),
    #             'dataset': 'mawps',
    #             'split': 'train' if i < int(0.8 * len(data)) else 'test'
    #         })
    #     df = pd.DataFrame(problems)
    #     train_df = df[df['split'] == 'train']
    #     test_df = df[df['split'] == 'test']
    #     train_df.to_csv('data/mawps_train.csv', index=False)
    #     test_df.to_csv('data/mawps_test.csv', index=False)
    #     print(f'✅ MAWPS: {len(train_df)} train, {len(test_df)} test samples')
    #     return True
    # except Exception as e:
    #     print(f'❌ MAWPS download failed: {e}')
    #     return False




In [25]:
def download_gsm8k():
    try:
        from datasets import load_dataset
        print("Downloading GSM8K dataset...")
        ds = load_dataset("gsm8k", name = "main")
        train_df = pd.DataFrame(ds["train"])
        test_df = pd.DataFrame(ds["test"])
        train_df.to_csv('gsm8k_train.csv', index=False)
        test_df.to_csv('gsm8k_test.csv', index=False)
        print(f'✅ GSM8K: {len(train_df)} train, {len(test_df)} test samples')
        return True
    except Exception as e:
        print(f'❌ GSM8K download failed: {e}')
        return False

In [26]:
from pickle import load
def download_mathqa():
    try:
        from datasets import load_dataset
        print("Downloading MathQA dataset...")
        ds = load_dataset("math_qa")
        train_df = pd.DataFrame(ds["train"])
        test_df = pd.DataFrame(ds["test"])
        validation_df = pd.DataFrame(ds["validation"])
        train_df.to_csv('mathqa_train.csv', index=False)
        test_df.to_csv('mathqa_test.csv', index=False)
        validation_df.to_csv('mathqa_validation.csv', index=False)
        print(f'✅ MathQA: {len(train_df)} train, {len(test_df)} test samples')
        return True
    except Exception as e:
        print(f'❌ MathQA download failed: {e}')
        return False

In [27]:
# Download SVAMP
def download_svamp():
    try:
        from datasets import load_dataset
        print("Downloading SVAMP dataset...")
        ds = load_dataset("ChilleD/SVAMP")
        train_df= pd.DataFrame(ds["train"])
        test_df = pd.DataFrame(ds["test"])
        train_df.to_csv('svamp_train.csv', index=False)
        test_df.to_csv('svamp_test.csv', index=False)
        print(f'✅ SVAMP: {len(train_df)} train, {len(test_df)} test samples')
        return True,
    except Exception as e:
        print(f'❌ SVAMP download failed: {e}')
        return False

In [28]:
# Make sure data directory exists
#Path('data').mkdir(exist_ok=True)
import tqdm

tqdm.tqdm = lambda *args, **kwargs: iter(args[0])  # disables tqdm progress bars

print("🚀 Starting dataset download...")
success_count = 0

if download_gsm8k():
    success_count += 1
if download_mathqa():
    success_count += 1
# if download_mawps():
#     success_count += 1
if download_svamp():
    success_count += 1

print(f"\n🎉 Successfully downloaded {success_count}/3 datasets")
print("Now you have real data instead of sample data!")

🚀 Starting dataset download...


Using the latest cached version of the dataset since math_qa couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\dlaev\.cache\huggingface\datasets\math_qa\default\0.1.0\67fc1cc5d22b185002c6fd16e19e4d5215eae01fb04d656bed83204ba6ee55ff (last modified on Tue Jul  8 17:10:32 2025).


✅ GSM8K: 7473 train, 1319 test samples
✅ MathQA: 29837 train, 2985 test samples
✅ SVAMP: 700 train, 300 test samples

🎉 Successfully downloaded 3/3 datasets
Now you have real data instead of sample data!


In [29]:
gsm8k_test_df = pd.read_csv('gsm8k_test.csv')
gsm8k_train_df = pd.read_csv('gsm8k_train.csv')

In [30]:
gsm8k_train_df.head()

,question,answer
0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...
1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...
2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $<<..."
3,"Julie is reading a 120-page book. Yesterday, s...",Maila read 12 x 2 = <<12*2=24>>24 pages today....
4,James writes a 3-page letter to 2 different fr...,He writes each friend 3*2=<<3*2=6>>6 pages a w...


In [31]:
gsm8k_test_df.head()

,question,answer
0,Janet’s ducks lay 16 eggs per day. She eats th...,Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eg...
1,A robe takes 2 bolts of blue fiber and half th...,It takes 2/2=<<2/2=1>>1 bolt of white fiber\nS...
2,Josh decides to try flipping a house. He buys...,The cost of the house and repairs came out to ...
3,James decides to run 3 sprints 3 times a week....,He sprints 3*3=<<3*3=9>>9 times\nSo he runs 9*...
4,"Every day, Wendi feeds each of her chickens th...","If each chicken eats 3 cups of feed per day, t..."


In [32]:
mathqa_test_df = pd.read_csv('mathqa_test.csv')
mathqa_train_df = pd.read_csv('mathqa_train.csv')
mathqa_validation_df = pd.read_csv('mathqa_validation.csv')

In [33]:
mathqa_train_df.head()

,Problem,Rationale,options,correct,annotated_formula,linear_formula,category
0,the banker ' s gain of a certain sum due 3 yea...,"""explanation : t = 3 years r = 10 % td = ( bg ...","a ) rs . 400 , b ) rs . 300 , c ) rs . 500 , d...",a,"divide(multiply(const_100, divide(multiply(36,...","multiply(n2,const_100)|multiply(n0,n1)|divide(...",gain
1,average age of students of an adult school is ...,"""explanation : let the original no . of studen...","a ) 1200 , b ) 120 , c ) 360 , d ) 240 , e ) n...",d,"multiply(divide(subtract(multiply(add(32, 4), ...","add(n2,n3)|multiply(n1,n2)|multiply(n1,#0)|sub...",general
2,sophia finished 2 / 3 of a book . she calculat...,let xx be the total number of pages in the boo...,"a ) 229 , b ) 270 , c ) 877 , d ) 266 , e ) 281",b,"divide(90, subtract(const_1, divide(2, 3)))","divide(n0,n1)|subtract(const_1,#0)|divide(n2,#1)",general
3,120 is what percent of 50 ?,"""50 * x = 120 - - > x = 2.4 - - > 2.4 expresse...","a ) 5 % , b ) 240 % , c ) 50 % , d ) 2 % , e )...",b,"multiply(divide(120, 50), const_100)","divide(n0,n1)|multiply(#0,const_100)|",gain
4,there are 10 girls and 20 boys in a classroom ...,"if girls is 10 and boys is 20 , then 10 / 20 ....","a ) 1 / 2 , b ) 1 / 3 , c ) 1 / 5 , d ) 10 / 3...",a,"divide(10, 20)","divide(n0,n1)",other


In [34]:
mathqa_test_df.head()

,Problem,Rationale,options,correct,annotated_formula,linear_formula,category
0,a shopkeeper sold an article offering a discou...,"""giving no discount to customer implies sellin...","a ) 38 , b ) 27.675 , c ) 30 , d ) data inadeq...",a,"subtract(divide(multiply(add(const_100, 31.1),...","add(n1,const_100)|subtract(const_100,n0)|multi...",gain
1,what will be the difference between simple and...,"""s . i . = ( 1000 * 14 * 4 ) / 100 = rs . 560 ...","a ) 129 , b ) 130 , c ) 124 , d ) 133 , e ) 145",a,"subtract(subtract(multiply(1000, power(add(div...","divide(n0,const_100)|add(#0,const_1)|multiply(...",gain
2,there are 28 stations between hyderabad and ba...,"""the total number of stations = 30 from 30 sta...","a ) 156 , b ) 167 , c ) 870 , d ) 352 , e ) 380",c,"multiply(add(28, const_1), add(add(28, const_1...","add(n0,const_1)|add(#0,const_1)|multiply(#0,#1)|",physics
3,the present population of a town is 3888 . pop...,"""p = 3888 r = 20 % required population of town...","a ) 2500 , b ) 2100 , c ) 3500 , d ) 3600 , e ...",e,"add(3888, divide(multiply(3888, 20), const_100))","multiply(n0,n1)|divide(#0,const_100)|add(n0,#1)|",gain
4,the triplicate ratio of 1 : 9 is ?,"""13 : 93 = 1 : 729 answer : e""","a ) 1 : 0 , b ) 1 : 8 , c ) 1 : 7 , d ) 1 : 2 ...",e,"divide(power(const_2.0, 9), power(const_3.0, 9))","power(const_2.0,n1)|power(const_3.0,n1)|divide...",other


In [35]:
mathqa_validation_df.head()

,Problem,Rationale,options,correct,annotated_formula,linear_formula,category
0,a multiple choice test consists of 4 questions...,"""5 choices for each of the 4 questions , thus ...","a ) 24 , b ) 120 , c ) 625 , d ) 720 , e ) 1024",c,"power(5, 4)","power(n1,n0)|",general
1,a 3 - digit positive integer is chosen at rand...,p ( three odd digits ) = 5 / 9 * 5 / 10 * 5 / ...,"a ) 1 / 2 , b ) 31 / 36 , c ) 49 / 54 , d ) 7 ...",b,"subtract(const_1, power(divide(const_1, const_...","divide(const_1,const_2)|power(#0,n0)|subtract(...",general
2,if x and y are positive integers and 7 + x + y...,"""try each answer choices . for a : 7 + 3 + xy ...","a ) 3 , b ) 5 , c ) 6 , d ) 8 , e ) 9",c,"add(21, const_1)","add(n1,const_1)|",general
3,the hcf and lcm of two numbers m and n are res...,"answer we have , m x n = 6 x 210 = 1260 ∴ 1 / ...","a ) 1 / 35 , b ) 3 / 35 , c ) 5 / 37 , d ) 2 /...",d,"divide(72, multiply(6, 210))","multiply(n0,n1)|divide(n2,#0)",general
4,"in a kilometer race , a beats b by 48 meters o...","""time taken by b run 1000 meters = ( 1000 * 12...","a ) 238 sec , b ) 190 sec , c ) 667 sec , d ) ...",a,"subtract(divide(multiply(const_1, const_1000),...","divide(n0,n1)|multiply(const_1,const_1000)|div...",physics


In [36]:
svamp_test_df = pd.read_csv('svamp_test.csv')
svamp_train_df = pd.read_csv('svamp_train.csv')

In [37]:
svamp_test_df.head()

,ID,Body,Question,Equation,Answer,Type,question_concat
0,chal-736,Winter is almost here and most animals are mig...,How many more bird families flew away to afric...,( 62.0 - 35.0 ),27,Subtraction,Winter is almost here and most animals are mig...
1,chal-162,Paige raised 7 goldfish and 12 catfish in the ...,How many fishes disappeared?,( ( 7.0 + 12.0 ) - 15.0 ),4,Subtraction,Paige raised 7 goldfish and 12 catfish in the ...
2,chal-349,Marco and his dad went strawberry picking. Tog...,How much did his dad's strawberries weigh now?,( ( 22.0 - 36.0 ) + 30.0 ),16,Addition,Marco and his dad went strawberry picking. Tog...
3,chal-390,Debby bought 200 water bottles and 256 soda bo...,How many days would the soda bottles last?,( 256.0 / 4.0 ),64,Common-Division,Debby bought 200 water bottles and 256 soda bo...
4,chal-781,There were 106 dollars in Olivia's wallet. Aft...,How much did she spend at the supermarket?,( ( 106.0 - 26.0 ) - 49.0 ),31,Subtraction,There were 106 dollars in Olivia's wallet. Aft...


In [38]:
svamp_train_df.head()

,ID,Body,Question,Equation,Answer,Type,question_concat
0,chal-777,There are 87 oranges and 290 bananas in Philip...,How big is each group of bananas?,( 290.0 / 2.0 ),145,Common-Division,There are 87 oranges and 290 bananas in Philip...
1,chal-508,Marco and his dad went strawberry picking. Mar...,How much did Marco's strawberries weigh?,( 30.0 - 11.0 ),19,Subtraction,Marco and his dad went strawberry picking. Mar...
2,chal-896,Edward spent $ 6 to buy 2 books each book cost...,How much did each book cost?,( 6.0 / 2.0 ),3,Common-Division,Edward spent $ 6 to buy 2 books each book cost...
3,chal-923,Frank was reading through his favorite book. T...,How many pages are in each chapter?,( 594.0 / 3.0 ),198,Common-Division,Frank was reading through his favorite book. T...
4,chal-34,There were 78 dollars in Olivia's wallet. She ...,How much money does she have left?,( 78.0 - 15.0 ),63,Subtraction,There were 78 dollars in Olivia's wallet. She ...


In [39]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
from typing import List, Dict, Any, Tuple
import warnings
warnings.filterwarnings('ignore')

# Import custom data loader
#from data_loader import MathDatasetLoader

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
print("Libraries imported successfully!")


Libraries imported successfully!


In [40]:
datasets = {
    "GSM8K Test": gsm8k_test_df,
    "GSM8K Train": gsm8k_train_df,
    "MathQA Test": mathqa_test_df,
    "MathQA Train": mathqa_train_df,
    "MathQA Validation": mathqa_validation_df,
    "SVAMP Test": svamp_test_df,
    "SVAMP Train": svamp_train_df,
}

In [41]:
# Initialize loader and load all datasets
# loader = MathDatasetLoader()
# datasets = loader.get_all_datasets()

# Show summary
for name, df in datasets.items():
    print(f'{name}: {df.shape[0]} samples, {df.shape[1]} features')
    print(f'Columns: {list(df.columns)}')
    print()

GSM8K Test: 1319 samples, 2 features
Columns: ['question', 'answer']

GSM8K Train: 7473 samples, 2 features
Columns: ['question', 'answer']

MathQA Test: 2985 samples, 7 features
Columns: ['Problem', 'Rationale', 'options', 'correct', 'annotated_formula', 'linear_formula', 'category']

MathQA Train: 29837 samples, 7 features
Columns: ['Problem', 'Rationale', 'options', 'correct', 'annotated_formula', 'linear_formula', 'category']

MathQA Validation: 4475 samples, 7 features
Columns: ['Problem', 'Rationale', 'options', 'correct', 'annotated_formula', 'linear_formula', 'category']

SVAMP Test: 300 samples, 7 features
Columns: ['ID', 'Body', 'Question', 'Equation', 'Answer', 'Type', 'question_concat']

SVAMP Train: 700 samples, 7 features
Columns: ['ID', 'Body', 'Question', 'Equation', 'Answer', 'Type', 'question_concat']



In [42]:
def validate_dataset(df: pd.DataFrame) -> Dict[str, Any]:
    """Validate dataset with safe handling of list columns."""
    result = {}
    result['missing_values'] = df.isnull().sum().to_dict()

    # Handle duplicates more safely - exclude list columns
    try:
        # Find columns that are safe to check for duplicates
        safe_columns = []
        for col in df.columns:
            if df[col].dtype in ['object']:
                # Check if this column contains lists by sampling
                sample_val = df[col].dropna().iloc[0] if not df[col].dropna().empty else None
                if not isinstance(sample_val, (list, dict)):
                    safe_columns.append(col)
            else:
                safe_columns.append(col)

        if safe_columns:
            result['duplicates'] = df[safe_columns].duplicated().sum()
            result['duplicate_check_columns'] = safe_columns
        else:
            result['duplicates'] = "Cannot check - all columns contain complex objects"
            result['duplicate_check_columns'] = []
    except Exception as e:
        result['duplicates'] = f"Error checking duplicates: {str(e)}"
        result['duplicate_check_columns'] = []

    result['data_types'] = df.dtypes.apply(str).to_dict()
    result['shape'] = df.shape
    return result

# Validate all datasets
print("Validating datasets...")
validation_results = {name: validate_dataset(df) for name, df in datasets.items()}

import pprint
pprint.pprint(validation_results)


Validating datasets...
{'GSM8K Test': {'data_types': {'answer': 'object', 'question': 'object'},
                'duplicate_check_columns': ['question', 'answer'],
                'duplicates': 0,
                'missing_values': {'answer': 0, 'question': 0},
                'shape': (1319, 2)},
 'GSM8K Train': {'data_types': {'answer': 'object', 'question': 'object'},
                 'duplicate_check_columns': ['question', 'answer'],
                 'duplicates': 0,
                 'missing_values': {'answer': 0, 'question': 0},
                 'shape': (7473, 2)},
 'MathQA Test': {'data_types': {'Problem': 'object',
                                'Rationale': 'object',
                                'annotated_formula': 'object',
                                'category': 'object',
                                'correct': 'object',
                                'linear_formula': 'object',
                                'options': 'object'},
                 'duplicate_ch

In [43]:
# FIXED VERSION - Copy this into your notebook cell
def validate_dataset_fixed(df: pd.DataFrame) -> Dict[str, Any]:
    """Validate dataset with safe handling of list columns."""
    result = {}
    result['missing_values'] = df.isnull().sum().to_dict()

    # Handle duplicates safely by converting lists to strings
    try:
        # Create a copy for duplicate checking
        df_check = df.copy()

        # Convert list columns to strings for duplicate checking
        for col in df_check.columns:
            if df_check[col].dtype == 'object':
                # Check if column contains lists
                sample_val = df_check[col].dropna().iloc[0] if not df_check[col].dropna().empty else None
                if isinstance(sample_val, list):
                    df_check[col] = df_check[col].astype(str)

        result['duplicates'] = df_check.duplicated().sum()
    except Exception as e:
        result['duplicates'] = f"Error: {str(e)}"

    result['data_types'] = df.dtypes.apply(str).to_dict()
    result['shape'] = df.shape
    return result

# Use the fixed function
print("Validating datasets with fixed function...")
validation_results = {name: validate_dataset_fixed(df) for name, df in datasets.items()}

import pprint
pprint.pprint(validation_results)


Validating datasets with fixed function...
{'GSM8K Test': {'data_types': {'answer': 'object', 'question': 'object'},
                'duplicates': 0,
                'missing_values': {'answer': 0, 'question': 0},
                'shape': (1319, 2)},
 'GSM8K Train': {'data_types': {'answer': 'object', 'question': 'object'},
                 'duplicates': 0,
                 'missing_values': {'answer': 0, 'question': 0},
                 'shape': (7473, 2)},
 'MathQA Test': {'data_types': {'Problem': 'object',
                                'Rationale': 'object',
                                'annotated_formula': 'object',
                                'category': 'object',
                                'correct': 'object',
                                'linear_formula': 'object',
                                'options': 'object'},
                 'duplicates': 0,
                 'missing_values': {'Problem': 0,
                                    'Rationale': 0,
         

In [44]:
# Fix: Load custom dataset using MathDatasetLoader to resolve KeyError
from data_loader import MathDatasetLoader

# Initialize the loader and get the custom dataset
loader = MathDatasetLoader()
custom_df = loader.load_custom_math_dataset()

# Add the custom dataset to our existing datasets dictionary
datasets['custom'] = custom_df

print(f"✅ Added custom dataset: {custom_df.shape[0]} samples, {custom_df.shape[1]} features")
print(f"Custom dataset columns: {list(custom_df.columns)}")


Custom dataset not found. Creating sample data...
✅ Added custom dataset: 5 samples, 9 features
Custom dataset columns: ['problem_id', 'problem_text', 'difficulty_level', 'subject', 'solution_steps', 'final_answer', 'symbolic_complexity', 'step_count', 'dataset']


In [45]:
# Test: Verify that all datasets are properly loaded
print("Available datasets:")
for name in datasets.keys():
    print(f"  - {name}")

print(f"\nTotal datasets loaded: {len(datasets)}")


Available datasets:
  - GSM8K Test
  - GSM8K Train
  - MathQA Test
  - MathQA Train
  - MathQA Validation
  - SVAMP Test
  - SVAMP Train
  - custom

Total datasets loaded: 8


In [46]:
# FIXED VERSION - Safe data cleaning for datasets with list columns
df = datasets['custom'].copy()

# Clean text fields first
def clean_text(text):
    if pd.isna(text): return ''
    text = re.sub(r'\s+', ' ', str(text)).strip()
    return text

df['problem_text'] = df['problem_text'].apply(clean_text)

# Safe duplicate removal - only check specific columns that don't contain lists
def safe_drop_duplicates(df):
    """Safely remove duplicates by only checking non-list columns."""
    try:
        # Identify columns that are safe to check for duplicates
        safe_columns = []
        for col in df.columns:
            if df[col].dtype == 'object':
                # Check if column contains lists
                sample_val = df[col].dropna().iloc[0] if not df[col].dropna().empty else None
                if not isinstance(sample_val, list):
                    safe_columns.append(col)
            else:
                safe_columns.append(col)

        print(f"Checking duplicates on columns: {safe_columns}")

        if safe_columns:
            # Only check duplicates on safe columns
            df_clean = df.drop_duplicates(subset=safe_columns)
        else:
            print("No safe columns for duplicate checking - skipping duplicate removal")
            df_clean = df.copy()

        return df_clean
    except Exception as e:
        print(f"Error in duplicate removal: {e}")
        return df


In [47]:
# Initialize loader and load all datasets
loader = MathDatasetLoader()
datasets = loader.get_all_datasets()

# Show summary
for name, df in datasets.items():
    print(f'{name}: {df.shape[0]} samples, {df.shape[1]} features')
    print(f'Columns: {list(df.columns)}')
    print()


Loaded GSM8K train dataset from local file
Loaded GSM8K test dataset from local file
Loaded MathQA train dataset from local file
Loaded MathQA test dataset from local file
Loaded MAWPS train dataset from local file
Loaded MAWPS test dataset from local file
Custom dataset not found. Creating sample data...
gsm8k_train: 7473 samples, 7 features
Columns: ['question', 'answer', 'solution', 'difficulty', 'category', 'dataset', 'split']

gsm8k_test: 1319 samples, 7 features
Columns: ['question', 'answer', 'solution', 'difficulty', 'category', 'dataset', 'split']

mathqa_train: 23869 samples, 7 features
Columns: ['Problem', 'Rationale', 'correct', 'options', 'category', 'dataset', 'split']

mathqa_test: 5968 samples, 7 features
Columns: ['Problem', 'Rationale', 'correct', 'options', 'category', 'dataset', 'split']

mawps_train: 8 samples, 7 features
Columns: ['sQuestion', 'lSolutions', 'lEquations', 'iIndex', 'category', 'dataset', 'split']

mawps_test: 2 samples, 7 features
Columns: ['sQuest

In [48]:
def validate_dataset_fixed(df):
    result = {}
    result['missing_values'] = df.isnull().sum().to_dict()

    # Handle duplicates safely by converting lists to strings
    try:
        df_check = df.copy()
        for col in df_check.columns:
            if df_check[col].dtype == 'object':
                sample_val = df_check[col].dropna().iloc[0] if not df_check[col].dropna().empty else None
                if isinstance(sample_val, list):
                    df_check[col] = df_check[col].astype(str)
        result['duplicates'] = df_check.duplicated().sum()
    except Exception as e:
        result['duplicates'] = f"Error: {str(e)}"

    result['data_types'] = df.dtypes.apply(str).to_dict()
    result['shape'] = df.shape
    return result

# Use the fixed function
validation_results = {name: validate_dataset_fixed(df) for name, df in datasets.items()}
import pprint
pprint.pprint(validation_results)

{'custom': {'data_types': {'dataset': 'object',
                           'difficulty_level': 'object',
                           'final_answer': 'object',
                           'problem_id': 'object',
                           'problem_text': 'object',
                           'solution_steps': 'object',
                           'step_count': 'int64',
                           'subject': 'object',
                           'symbolic_complexity': 'int64'},
            'duplicates': 0,
            'missing_values': {'dataset': 0,
                               'difficulty_level': 0,
                               'final_answer': 0,
                               'problem_id': 0,
                               'problem_text': 0,
                               'solution_steps': 0,
                               'step_count': 0,
                               'subject': 0,
                               'symbolic_complexity': 0},
            'shape': (5, 9)},
 'gsm8k_test': {

In [49]:
# FIXED VERSION - Safe data cleaning for datasets with list columns
# Check if custom dataset exists, if not skip this section
if 'custom' in datasets:
    df = datasets['custom'].copy()
    
    # Clean text fields first
    def clean_text(text):
        if pd.isna(text): return ''
        text = re.sub(r'\s+', ' ', str(text)).strip()
        return text

    if 'problem_text' in df.columns:
        df['problem_text'] = df['problem_text'].apply(clean_text)

    # Safe duplicate removal - only check specific columns that don't contain lists
    def safe_drop_duplicates(df):
        """Safely remove duplicates by only checking non-list columns."""
        try:
            # Identify columns that are safe to check for duplicates
            safe_columns = []
            for col in df.columns:
                if df[col].dtype == 'object':
                    # Check if column contains lists
                    sample_val = df[col].dropna().iloc[0] if not df[col].dropna().empty else None
                    if not isinstance(sample_val, list):
                        safe_columns.append(col)
                else:
                    safe_columns.append(col)

            print(f"Checking duplicates on columns: {safe_columns}")

            if safe_columns:
                # Only check duplicates on safe columns
                df_clean = df.drop_duplicates(subset=safe_columns)
            else:
                print("No safe columns for duplicate checking - skipping duplicate removal")
                df_clean = df.copy()

            return df_clean
        except Exception as e:
            print(f"Error in duplicate removal: {e}")
            return df

    # Apply safe duplicate removal
    df = safe_drop_duplicates(df)

    # Remove completely empty rows
    df = df.dropna(how='all')

    print(f"Dataset shape after cleaning: {df.shape}")
    print(df.head())
else:
    print("⚠️ Custom dataset not found in datasets dictionary.")
    print("The custom dataset should be available through the MathDatasetLoader.")
    print("Try running this instead:")
    print("from data_loader import MathDatasetLoader")
    print("loader = MathDatasetLoader()")
    print("df = loader.load_custom_math_dataset()")

# Apply safe duplicate removal
df = safe_drop_duplicates(df)

# Remove completely empty rows
df = df.dropna(how='all')

print(f"Dataset shape after cleaning: {df.shape}")
print(df.head())

Checking duplicates on columns: ['problem_id', 'problem_text', 'difficulty_level', 'subject', 'final_answer', 'symbolic_complexity', 'step_count', 'dataset']
Dataset shape after cleaning: (5, 9)
  problem_id                                       problem_text  \
0       P001                           Solve for x: 2x + 5 = 13   
1       P002         Find the derivative of f(x) = x^2 + 3x + 1   
2       P003  Calculate the area under the curve y = x^2 fro...   
3       P004     Prove that the sum of two even numbers is even   
4       P005     Find the limit as x approaches 0 of (sin(x))/x   

  difficulty_level        subject  \
0            basic        algebra   
1     intermediate       calculus   
2         advanced       calculus   
3     intermediate  number_theory   
4         advanced       calculus   

                                      solution_steps     final_answer  \
0  [Subtract 5 from both sides: 2x = 8, Divide by...            x = 4   
1  [Apply power rule: d/dx(x^2) =

In [50]:
# Example: Extract text length and operator count
def extract_features(text):
    features = {}
    features['text_length'] = len(text)
    features['operator_count'] = sum(text.count(op) for op in ['+', '-', '*', '/', '=', '^'])
    return features

features_df = df['problem_text'].apply(extract_features).apply(pd.Series)
df = pd.concat([df, features_df], axis=1)

print(df.head())


  problem_id                                       problem_text  \
0       P001                           Solve for x: 2x + 5 = 13   
1       P002         Find the derivative of f(x) = x^2 + 3x + 1   
2       P003  Calculate the area under the curve y = x^2 fro...   
3       P004     Prove that the sum of two even numbers is even   
4       P005     Find the limit as x approaches 0 of (sin(x))/x   

  difficulty_level        subject  \
0            basic        algebra   
1     intermediate       calculus   
2         advanced       calculus   
3     intermediate  number_theory   
4         advanced       calculus   

                                      solution_steps     final_answer  \
0  [Subtract 5 from both sides: 2x = 8, Divide by...            x = 4   
1  [Apply power rule: d/dx(x^2) = 2x, Apply const...   f'(x) = 2x + 3   
2  [Set up integral: ∫₀² x² dx, Apply power rule:...       Area = 8/3   
3  [Let a = 2k, b = 2m for integers k,m, a + b = ...  The sum is even   
4  [Use L

In [51]:
# Save cleaned and feature-engineered data
df.to_csv('preprocessed_custom_math_data.csv', index=False)
print("Saved preprocessed data to 'preprocessed_custom_math_data.csv'")


Saved preprocessed data to 'preprocessed_custom_math_data.csv'
